In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
df = pd.read_csv('data/merge_xy.csv',encoding='utf-8-sig')
df

,기준_년분기_코드,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수,상권_라벨,서비스_라벨,...,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,변화_지표,행정동_라벨,동별_임대료,latitude,longitude
0,20234,583,583,0,0,1,4,0,262,64,...,1354774,2557324,2157729,2607029,4526402,0,49,112919,37.514004,126.940269
1,20224,608,608,0,1,0,1,0,262,64,...,1289771,2413132,2106826,2297953,2956300,0,49,112919,37.514004,126.940269
2,20211,612,612,1,5,1,4,0,262,64,...,1039750,2502903,2582133,2811751,3284720,0,49,112919,37.514004,126.940269
3,20214,1873,1875,1,18,1,22,2,1070,59,...,206615,291151,351842,298119,206427,0,360,117210,37.533273,126.961560
4,20214,614,614,0,1,1,4,0,262,64,...,1460974,2421794,2042804,2239717,2440004,0,49,112919,37.514004,126.940269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229410,20214,67,67,12,8,2,1,0,595,99,...,0,0,5,0,0,2,119,95279,37.477525,126.982073
229411,20211,5,5,20,1,0,0,0,236,39,...,0,20,0,0,0,0,28,109576,37.490081,126.886535
229412,20223,4,4,0,0,0,0,0,1118,76,...,0,0,118,0,0,1,296,133399,37.576507,127.006507
229413,20231,4,4,0,0,0,0,0,1118,76,...,0,0,47,0,0,0,296,133399,37.576507,127.006507


In [9]:
# 평균 계산
mean_sales = df['당월_매출_금액'].mean()

# 결과 출력
print("당월_매출_금액 평균:", mean_sales)

당월_매출_금액 평균: 865158748.3650396


### **전처리**

In [ ]:
# 기준 년분기 코드를 datetime으로 변환하는 함수
def convert_quarter_to_date(q_code):
    year = str(q_code)[:4]  # 년도 추출
    quarter = int(str(q_code)[4])  # 분기 추출
    month = (quarter - 1) * 3 + 1  # 분기를 월로 변환 (1, 4, 7, 10)
    return pd.Timestamp(year + '-' + str(month) + '-01')

# 데이터프레임 df에서 '기준_년분기_코드' 열을 변환
df['날짜'] = df['기준_년분기_코드'].apply(convert_quarter_to_date)

# 날짜를 인덱스로 설정
df.set_index('날짜', inplace=True)

# 상권_라벨이 126인 행 제외
df = df[df['상권_라벨'] != 126]

# 결측치가 50% 이상인 열 삭제
df = df.dropna(thresh=len(df) * 0.5, axis=1)

# 99% 백분위값 계산 (상권_라벨 + 서비스_라벨 조합별)
percentiles = df.groupby(['상권_라벨', '서비스_라벨'])['당월_매출_금액'].quantile(0.99).reset_index()
percentiles.columns = ['상권_라벨', '서비스_라벨', '99퍼센타일값']

# 이상치 값 대체 (상권 262의 서비스 64번, 상권 1070의 서비스 59번)
for (region, service), p99_value in percentiles.set_index(['상권_라벨', '서비스_라벨']).iterrows():
    mask = (df['상권_라벨'] == region) & (df['서비스_라벨'] == service)
    df.loc[mask & (df['당월_매출_금액'] > p99_value['99퍼센타일값']), '당월_매출_금액'] = p99_value['99퍼센타일값']

# 2차 이상치 처리: 평균 ± 2표준편차 기준 제거
grouped_stats = df.groupby('서비스_라벨')['당월_매출_금액'].agg(['mean', 'std']).reset_index()
grouped_stats.columns = ['서비스_라벨', '평균_매출', '표준편차']
grouped_stats['하한'] = (grouped_stats['평균_매출'] - 2 * grouped_stats['표준편차']).clip(lower=0)
grouped_stats['상한'] = grouped_stats['평균_매출'] + 2 * grouped_stats['표준편차']

# (262, 64), (1070, 59) 제외하고 이상치 제거
exclude_combinations = [(262, 64), (1070, 59)]

for _, row in grouped_stats.iterrows():
    service_label = row['서비스_라벨']
    lower_bound = row['하한']
    upper_bound = row['상한']
    
    # 해당 서비스 라벨에 대해 상권-서비스 조합 확인
    affected_rows = df[df['서비스_라벨'] == service_label]

    for region in affected_rows['상권_라벨'].unique():
        if (region, service_label) not in exclude_combinations:
            mask = (df['상권_라벨'] == region) & (df['서비스_라벨'] == service_label)
            df = df[~(mask & (
                (df['당월_매출_금액'] < lower_bound) |
                (df['당월_매출_금액'] > upper_bound)
            ))]
# 이상치 제거 후 평균 계산
filtered_mean = df['당월_매출_금액'].mean()
print("이상치 제거 후 평균 당월 매출 금액:", filtered_mean)

# 로그 변환
df['로그_당월_매출_금액'] = np.log1p(df['당월_매출_금액'])

# 결과 출력
print(df.head())


이상치 제거 후 평균 당월 매출 금액: 592568647.680186
            기준_년분기_코드  점포_수  유사_업종_점포_수  개업_율  개업_점포_수  폐업_률  폐업_점포_수  \
날짜                                                                      
2023-10-01      20234   583         583     0        0     1        4   
2022-10-01      20224   608         608     0        1     0        1   
2021-01-01      20211   612         612     1        5     1        4   
2021-10-01      20214  1873        1875     1       18     1       22   
2021-10-01      20214   614         614     0        1     1        4   

            프랜차이즈_점포_수  상권_라벨  서비스_라벨  ...  연령대_30_매출_건수  연령대_40_매출_건수  \
날짜                                     ...                               
2023-10-01           0    262      64  ...       2557324       2157729   
2022-10-01           0    262      64  ...       2413132       2106826   
2021-01-01           0    262      64  ...       2502903       2582133   
2021-10-01           2   1070      59  ...        291151        351842   
2021-

In [8]:
# 평균 계산
mean_sales = df['당월_매출_금액'].mean()

# 결과 출력
print("당월_매출_금액 평균:", mean_sales)

당월_매출_금액 평균: 955768085.1174092


### **Clustering**

In [11]:
# 상권 라벨과 서비스 라벨을 클러스터링하기 위한 데이터 준비
X_clustering = df[['상권_라벨', '서비스_라벨']]

# KMeans 클러스터링 수행
# KMeans 클러스터링 (상권 라벨 500개)
kmeans_location = KMeans(n_clusters=500, random_state=42)
df['상권_클러스터'] = kmeans_location.fit_predict(X_clustering[['상권_라벨']])

# KMeans 클러스터링 (서비스 라벨 60개)
kmeans_service = KMeans(n_clusters=60, random_state=42)
df['서비스_클러스터'] = kmeans_service.fit_predict(X_clustering[['서비스_라벨']])


# 클러스터링 결과 출력 (상위 10개 행)
print("클러스터링 결과:")
print(df[['상권_라벨', '서비스_라벨', '상권_클러스터', '서비스_클러스터']].head(10))


클러스터링 결과:
            상권_라벨  서비스_라벨  상권_클러스터  서비스_클러스터
날짜                                          
2023-10-01    262      64      317        46
2022-10-01    262      64      317        46
2021-01-01    262      64      317        46
2021-10-01   1070      59       36        50
2021-10-01    262      64      317        46
2022-04-01    262      64      317        46
2023-01-01    262      64      317        46
2022-01-01    262      64      317        46
2021-01-01   1070      59       36        50
2021-07-01   1070      59       36        50


### **데이터 생성**

In [12]:
# log 데이터 준비
log_data = df[['로그_당월_매출_금액']]
n_steps = 10  # 타임 스텝 설정

# 데이터 준비 함수
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step)])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

# 데이터셋 생성
X, y = create_dataset(log_data.values, time_step=n_steps)

# 클러스터 정보를 추가
clusters = df[['상권_클러스터', '서비스_클러스터']].values[n_steps:]  # 예측하는 시점의 클러스터 정보

# 차원 확인
print("X의 차원:", X.shape)  # X의 차원 확인
print("clusters의 차원:", clusters.shape)  # clusters의 차원 확인

# 클러스터 정보를 3D로 변환
clusters = np.repeat(clusters[:, np.newaxis, :], n_steps, axis=1)  # (샘플 수, 타임 스텝, 2)로 변환

# 모든 열을 포함하여 X를 생성
all_features = df.values[n_steps:]  # 모든 열을 포함한 데이터

# X의 차원을 3D로 변환 (타임 스텝을 맞추기 위해)
X = X.reshape(X.shape[0], X.shape[1], 1)  # (샘플 수, 타임 스텝, 1)

# 클러스터 정보를 추가
X = np.concatenate((X, clusters), axis=2)  # 기존 X와 클러스터 정보를 결합

# all_features를 3D로 변환 (샘플 수, 타임 스텝, 열 수)
all_features_3d = np.repeat(all_features[:, np.newaxis, :], n_steps, axis=1)

# 모든 열을 결합
X = np.concatenate((X, all_features_3d), axis=2)

# 생성된 X와 y의 형태 확인
print("X의 형태:", X.shape)  # X의 형태를 출력
print("y의 형태:", y.shape)  # y의 형태를 출력

X의 차원: (216758, 10, 1)
clusters의 차원: (216758, 2)
X의 형태: (216758, 10, 185)
y의 형태: (216758, 1)


In [13]:
# 데이터 스케일링
scaler = MinMaxScaler()
X_reshaped = X.reshape(-1, X.shape[-1])  # 2D로 변환
X_scaled = scaler.fit_transform(X_reshaped).reshape(X.shape)  # 다시 3D로 변환

# y 스케일링
y_scaled = MinMaxScaler().fit_transform(y.reshape(-1,1)).flatten()

In [14]:
# 데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# X_temp를 검증 데이터와 테스트 데이터로 다시 분할
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## **LSTM**

#### 확실한 코드는 아닙니다,,,

In [16]:
# LSTM 모델 정의
model = Sequential()

# Input 레이어 추가
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

# 첫 번째 LSTM 층
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.4))  # 드롭아웃 추가 (과적합 방지)

# 두 번째 LSTM 층
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))  # 드롭아웃 추가

# # 세 번째 LSTM 층
# model.add(LSTM(200, return_sequences=True))
# model.add(Dropout(0.4))  # 드롭아웃 추가

# 네  번째 LSTM 층
model.add(LSTM(64))
model.add(Dropout(0.4))  # 드롭아웃 추가

# 출력 층
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(X_train, y_train, epochs=100, batch_size=128)

# 예측 수행
y_pred_scaled = model.predict(X_test)

# NaN 및 inf 확인
print("y_pred_scaled contains NaN:", np.isnan(y_pred_scaled).any())
print("y_pred_scaled contains inf:", np.isinf(y_pred_scaled).any())

# 값 클리핑
y_pred_scaled = np.clip(y_pred_scaled, -1e10, 1e10)

# y_test를 2차원으로 reshape
y_test_reshaped = y_test.reshape(-1, 1)

# y_test의 NaN 및 inf 확인
print("y_test contains NaN:", np.isnan(y_test_reshaped).any())
print("y_test contains inf:", np.isinf(y_test_reshaped).any())

# # 값 클리핑
# y_test_reshaped = np.clip(y_test_reshaped, -1e10, 1e10)

# 역변환: Min-Max 스케일링 복원
y_pred_original_scaled = scaler.inverse_transform(np.tile(y_pred_scaled, (1, X_train.shape[2])))

# 예측 값의 로그 변환된 값을 역변환
y_pred_original = np.exp(y_pred_original_scaled + 1e-10)

# y_test도 역변환
y_test_original_scaled = scaler.inverse_transform(np.tile(y_test_reshaped, (1, X_train.shape[2])))
y_test_original = np.exp(y_test_original_scaled + 1e-10)

# 값 클리핑
y_pred_original = np.clip(y_pred_original, -1e10, 1e10)
y_test_original = np.clip(y_test_original, -1e10, 1e10)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))  
print("RMSE:", rmse)

# MAE 계산
mae = mean_absolute_error(y_test_original, y_pred_original) 
print("MAE:", mae)

# R² 계산
r_squared = r2_score(y_test_original, y_pred_original)  
print("R²:", r_squared)

Epoch 1/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 151s 105ms/step - loss: 0.0110
Epoch 2/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 145s 107ms/step - loss: 0.0017
Epoch 3/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 145s 107ms/step - loss: 3.5984e-04
Epoch 4/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 143s 105ms/step - loss: 1.3745e-04
Epoch 5/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 160s 118ms/step - loss: 1.2462e-04
Epoch 6/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 149s 110ms/step - loss: 1.1441e-04
Epoch 7/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 145s 107ms/step - loss: 1.0173e-04
Epoch 8/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 138s 102ms/step - loss: 9.9234e-05
Epoch 9/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 140s 103ms/step - loss: 1.0089e-04
Epoch 10/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 141s 104ms/step - loss: 9.5660e-05
Epoch 11/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 140s 104ms/step - loss: 8.6866e-05
Epoch 12/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 141s 104ms/step - loss: 8.7149e-05
Epoch 13/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 140s 103ms/step - loss: 8

C:\Users\user\AppData\Local\Temp\ipykernel_3424\2489165613.py:56: RuntimeWarning: overflow encountered in exp
  y_pred_original = np.exp(y_pred_original_scaled + 1e-10)
C:\Users\user\AppData\Local\Temp\ipykernel_3424\2489165613.py:60: RuntimeWarning: overflow encountered in exp
  y_test_original = np.exp(y_test_original_scaled + 1e-10)


RMSE: 18942986.650852785
MAE: 699361.4089352184
R²: 0.9833500038704971


## **GRU** 코드 없음다

## **DEEPAR+**

In [15]:
# DeepAR+ 스타일 모델 정의
def create_deep_ar_model(input_shape):
    model = keras.Sequential()

    # LSTM 층 추가
    model.add(layers.LSTM(256, return_sequences=True, input_shape=input_shape))
    model.add(layers.Dropout(0.4))

    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.4))

    model.add(layers.LSTM(64))
    model.add(layers.Dropout(0.4))

    # 예측을 위한 출력층
    model.add(layers.Dense(1))  # 예측 값
    return model

# 모델 생성
input_shape = (X_train.shape[1], X_train.shape[2])  # (타임 스텝 수, 특성 수)
model = create_deep_ar_model(input_shape)

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 훈련
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val))

# 예측 수행
y_pred_scaled = model.predict(X_test)

# 역변환: 원래 스케일로 복원
y_pred = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_pred_scaled), axis=1))[:, -1]
y_test_original = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test_original, y_pred))
print("RMSE:", rmse)

# MAE 계산
mae = mean_absolute_error(y_test_original, y_pred)
print("MAE:", mae)

# R² 계산
r_squared = r2_score(y_test_original, y_pred)
print("R²:", r_squared)

c:\Users\user\anaconda3\envs\study\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 170s 120ms/step - loss: 0.0117 - val_loss: 1.2138e-04
Epoch 2/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 157s 116ms/step - loss: 0.0017 - val_loss: 3.9195e-05
Epoch 3/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 161s 119ms/step - loss: 3.6424e-04 - val_loss: 4.3524e-05
Epoch 4/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 151s 112ms/step - loss: 1.4489e-04 - val_loss: 1.6889e-05
Epoch 5/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 164s 121ms/step - loss: 1.2218e-04 - val_loss: 6.0011e-05
Epoch 6/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 151s 112ms/step - loss: 1.1007e-04 - val_loss: 3.2847e-05
Epoch 7/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 151s 111ms/step - loss: 1.1285e-04 - val_loss: 3.1632e-05
Epoch 8/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 155s 115ms/step - loss: 9.5728e-05 - val_loss: 1.2605e-05
Epoch 9/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 150s 111ms/step - loss: 9.4526e-05 - val_loss: 4.5752e-05
Epoch 10/100
1355/1355 ━━━━━━━━━━━━━━━━━━━━ 150s 111ms/step - loss: 9.3535e-05 - val_loss: 1.4899e-

##### 참고

DeepAR+ 모델에서 **오차범위 (예: RMSE, MAE, 등)**이 0.2처럼 낮게 나온다면, 몇 가지 이유가 있을 수 있습니다. 
   1. 데이터의 품질 및 특성
데이터의 일관성: 3개년 4분기씩의 데이터는 시간에 따른 패턴이 명확하게 드러날 수 있기 때문에, 모델이 이런 패턴을 잘 학습할 수 있습니다. 특히 시계열 데이터에서 계절성(예: 분기마다 발생하는 패턴)이 뚜렷하면 모델이 이를 잘 잡아낼 수 있습니다.
짧은 시계열 데이터: 3년, 즉 12개의 시점만 가지고 있으면 데이터의 복잡성이 적고, 모델이 과적합(overfitting)되는 것을 방지할 수 있습니다. 이는 오히려 모델이 적합하게 잘 학습할 수 있는 환경을 제공할 수 있습니다.
좋은 스케일링 및 전처리: 데이터를 잘 전처리하고 스케일링했다면, 모델이 더 효율적으로 학습하고 예측할 수 있습니다.
   2. 모델의 성능
DeepAR+의 장점: DeepAR+와 같은 시계열 예측 모델은 시계열의 패턴을 잘 학습할 수 있는 구조입니다. 특히 DeepAR+는 시간적 의존성과 특징들 간의 관계를 잘 모델링할 수 있어 예측의 정확도가 높을 수 있습니다.
과적합 방지: 모델이 너무 복잡하지 않거나, 적절한 정규화 및 **드롭아웃(dropout)**을 사용했다면, 훈련 데이터에 과적합되지 않고 일반화가 잘 된 결과를 얻을 수 있습니다.
   3. 데이터의 양과 다양성
데이터의 다양성 부족: 데이터가 3개년 4분기씩밖에 없다면, 모델이 학습할 수 있는 정보의 양이 제한적입니다. 데이터가 적을수록 모델이 패턴을 과도하게 학습할 수 있기 때문에 일반화가 잘되지 않을 수 있습니다. 하지만 그만큼 예측의 오차가 줄어들 수도 있습니다.
데이터의 충분성: 반면, 데이터가 간결하고 충분히 유의미한 패턴을 포함하고 있으면 모델이 쉽게 패턴을 찾을 수 있고, 오차가 낮게 나올 수 있습니다.
   4. 평가 지표의 선택
RMSE나 MAE 같은 평가 지표가 낮다는 것은 모델이 예측을 잘 수행하고 있다는 신호입니다. 그러나 때때로 단기 예측에서 모델이 단기적 패턴에 맞춰 잘 맞춰지는 경우가 많아, 오차가 적게 나올 수 있습니다. 예를 들어, 계절성 또는 분기별 변화를 모델이 잘 포착했다면, 예측 오차가 매우 적을 수 있습니다.
   5. 테스트 세트와 검증 세트
테스트 데이터가 매우 잘 설정되었거나, 모델이 과도하게 잘 맞춰졌을 수 있습니다. 예를 들어, 테스트 데이터가 훈련 데이터와 비슷하거나 과도하게 잘 조정된 경우 모델이 높은 정확도를 보일 수 있습니다.   
   6. **결론**:
오차범위가 낮은 이유는 여러 가지가 있을 수 있습니다. 데이터의 특성이나 모델의 학습 방식, 평가 지표 등 다양한 요인이 복합적으로 작용한 결과일 수 있습니다. 데이터가 잘 나왔다면 모델이 잘 예측할 수 있고, 그만큼 DeepAR+ 모델이 패턴을 잘 학습했기 때문에 오차가 낮을 수 있습니다. 다만, 테스트 데이터와 훈련 데이터 간의 차이를 고려하여 과적합되지 않도록 유의하는 것도 중요합니다.